#Install and Config LingX

In [1]:
!git clone https://github.com/ContentSide/lingx.git
%cd lingx
!pip install -e .
!python -m lingx.utils.download_lang_models

Cloning into 'lingx'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 362 (delta 184), reused 347 (delta 169), pack-reused 0
Receiving objects: 100% (362/362), 630.50 KiB | 13.13 MiB/s, done.
Resolving deltas: 100% (184/184), done.
/content/lingx
Obtaining file:///content/lingx
     |████████████████████████████████| 334 kB 5.1 MB/s 
  Created wheel for treelib: filename=treelib-1.6.1-py3-none-any.whl size=18386 sha256=0a1eff4b852cf134c80d04ce46ff3a96f16a4bc06ed84753ad9a98e011bce668
  Stored in directory: /root/.cache/pip/wheels/89/be/94/2c6d949ce599d1443426d83ba4dc93cd35c0f4638260930a53
Successfully built treelib
  Running setup.py develop for lingx
2021-07-21 00:48:28 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | partut  |
| mwt       | partut  |
| pos       | partut  |
| lemma     | partut

# Import LingX and NLP Objects

In [3]:
from lingx.core.lang_model import get_nlp_object


from lingx.utils.lx import compare_sentences_lx
from lingx.utils.lx import get_sentence_lx


from lingx.metrics.monolingual.le import get_le_score
from lingx.metrics.monolingual.mbn import get_mbn_score
from lingx.metrics.bilingual.bcr import get_bcr_score

from lingx.utils.critt.aligner import generate_alignment_pipelines

from lingx.utils.critt.tables import readTPRDBtables
from lingx.utils.critt.tables import convert_st2segment , convert_tt2segment
from lingx.utils.critt.tables import expand_table_psycholingual , expand_table_monolingual , expand_table_bilingual
from lingx.utils.critt.tables import merge_st_tt , adhoc_cleanup



nlp_en = get_nlp_object("en", use_critt_tokenization = True, package="partut")
nlp_zh = get_nlp_object("zh", use_critt_tokenization = True)

2021-07-21 00:52:39 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | partut  |
| pos       | partut  |
| lemma     | partut  |
| depparse  | partut  |

2021-07-21 00:52:39 INFO: Use device: gpu
2021-07-21 00:52:39 INFO: Loading: tokenize
2021-07-21 00:52:39 INFO: Loading: pos
2021-07-21 00:52:46 INFO: Loading: lemma
2021-07-21 00:52:46 INFO: Loading: depparse
2021-07-21 00:52:47 INFO: Done loading processors!
2021-07-21 00:52:47 INFO: "zh" is an alias for "zh-hans"
2021-07-21 00:52:47 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsdsimp |
| pos       | gsdsimp |
| lemma     | gsdsimp |
| depparse  | gsdsimp |

2021-07-21 00:52:47 INFO: Use device: gpu
2021-07-21 00:52:47 INFO: Loading: tokenize
2021-07-21 00:52:47 INFO: Loading: pos
2021-07-21 00:52:47 INFO: Loading: lemma
2021-07-21 00:52:48 INFO: Loading: depparse
2021-07-21 00:52

#Calculate the Complexities

In [10]:
path_to_tprdb = "/content/lingx/resources/TPRDB/EN-ZH_IMBst18/"

df_sg = readTPRDBtables(['Tables/'], "*sg", verbose=1, path=path_to_tprdb)
df_st = readTPRDBtables(['Tables/'], "*st", verbose=1, path=path_to_tprdb)
df_tt = readTPRDBtables(['Tables/'], "*tt", verbose=1, path=path_to_tprdb)

alignments_offset = generate_alignment_pipelines(df_st, df_tt)

analysis_st = convert_st2segment(df_st)
analysis_tt = convert_tt2segment(df_tt)

analysis_st = expand_table_psycholingual(analysis_st, nlp_en, token_column="SToken")
analysis_tt = expand_table_psycholingual(analysis_tt, nlp_zh, token_column="TToken")

analysis_st =  expand_table_monolingual(analysis_st, nlp_en, token_column="SToken")
analysis_tt =  expand_table_monolingual(analysis_tt, nlp_zh, token_column="TToken")

analysis_st_tt = merge_st_tt(analysis_st, analysis_tt, alignments_offset)

# analysis_st_tt = adhoc_cleanup(analysis_st_tt)

analysis_st_tt = expand_table_bilingual(analysis_st_tt, nlp_en, nlp_zh)

print(analysis_st_tt)
print(analysis_st_tt.columns)

Reading: /content/lingx/resources/TPRDB/EN-ZH_IMBst18/Tables/*sg
Reading: /content/lingx/resources/TPRDB/EN-ZH_IMBst18/Tables/*st
Reading: /content/lingx/resources/TPRDB/EN-ZH_IMBst18/Tables/*tt


/content/lingx/lingx/utils/critt/aligner.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Part_Text_TTseg"] = df.apply(lambda x : str(x['Part']) + "_" + str(x['Text']) + "_" + str(x['TTseg']), axis=1)
/content/lingx/lingx/utils/critt/aligner.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tt_filtered["Part_Text_TTseg"] = df_tt_filtered.apply(lambda x : str(x['Part']) + "_" + str(x['Text']) + "_" + str(x['TTseg']), axis=1)
/content/lingx/lingx/utils/critt/aligner.py:36: SettingWithCopyWarning:

    Part  Text  ...  bilingual_score_sum_mean_max bilingual_score_sum_mean_mean
0    P00     3  ...                      1.500000                      1.222222
1    P00     3  ...                      3.000000                      3.000000
2    P00     3  ...                      0.750000                      0.750000
3    P00     3  ...                      1.666667                      1.430556
4    P00     3  ...                      4.400000                      2.035034
..   ...   ...  ...                           ...                           ...
195  P09     6  ...                      1.666667                      1.666667
196  P09     6  ...                      0.875000                      0.875000
197  P09     6  ...                      2.000000                      1.625000
198  P09     6  ...                      0.000000                           NaN
199  P09     6  ...                      0.852632                      0.592982

[200 rows x 30 columns]
Index(['Part', 

#Connect the Metric Results to Human-level Analysis

In [11]:
import pandas as pd

file_path = "/content/lingx/resources/TPRDB/EN-ZH_IMBst18/HumanEvaluations/errors.csv"
errors = pd.read_csv(file_path, index_col=False)

analysis_st_tt["SessionSeg"] = analysis_st_tt.apply(lambda x :x["Part"] + "_Ist" + str(x["Text"])+ "_" + str(x["TTseg"]), axis=1)


analysis_st_tt = pd.merge(analysis_st_tt,
                                  errors,
                                  on=["SessionSeg"],
                                  how='left')

analysis_st_tt.columns

Index(['Part', 'Text', 'TTseg', 'TToken', 'IDT_MAX', 'IDT_MEAN', 'IDT_SUM',
       'DLT_MAX', 'DLT_MEAN', 'DLT_SUM', 'IDT_DLT_MAX', 'IDT_DLT_MEAN',
       'IDT_DLT_SUM', 'LE_MEAN', 'LE_MAX', 'LE_SUM', 'MBN_MEAN', 'MBN_MAX',
       'MBN_SUM', 'SToken', 'alignment', 'bilingual_score_sum_sum_sum',
       'bilingual_score_sum_sum_max', 'bilingual_score_sum_sum_mean',
       'bilingual_score_sum_max_sum', 'bilingual_score_sum_max_max',
       'bilingual_score_sum_max_mean', 'bilingual_score_sum_mean_sum',
       'bilingual_score_sum_mean_max', 'bilingual_score_sum_mean_mean',
       'SessionSeg', 'Any', 'Accuracy', 'Fluency', 'Style', 'Critical',
       'Minor'],
      dtype='object')

In [12]:
table = analysis_st_tt[[
                        'IDT_MAX', 'IDT_MEAN', 'IDT_SUM',
                        'DLT_MAX', 'DLT_MEAN', 'DLT_SUM', 'IDT_DLT_MAX', 'IDT_DLT_MEAN',
                        'IDT_DLT_SUM', 'LE_MEAN', 'LE_MAX', 'LE_SUM', 'MBN_MEAN', 'MBN_MAX',
                        'MBN_SUM', 'SToken', 'alignment', 'bilingual_score_sum_sum_sum',
                        'bilingual_score_sum_sum_max', 'bilingual_score_sum_sum_mean',
                        'bilingual_score_sum_max_sum', 'bilingual_score_sum_max_max',
                        'bilingual_score_sum_max_mean', 'bilingual_score_sum_mean_sum',
                        'bilingual_score_sum_mean_max', 'bilingual_score_sum_mean_mean',
                        'SessionSeg', 'Any', 'Accuracy', 'Fluency', 'Style', 'Critical',
                        'Minor'
                        ]]

In [14]:
report = table.corr(method="spearman")

In [18]:
report.to_csv("report.csv")

#Correlation Matrix

In [20]:
report[['Any','Accuracy','Fluency','Style','Critical','Minor']]

,Any,Accuracy,Fluency,Style,Critical,Minor
IDT_MAX,0.484372,0.341186,0.360054,0.039784,0.316224,0.353183
IDT_MEAN,0.516977,0.370778,0.342827,0.008117,0.357886,0.322023
IDT_SUM,0.615684,0.426849,0.450015,0.144323,0.396922,0.497227
DLT_MAX,0.417281,0.253904,0.421119,0.076654,0.275013,0.329394
DLT_MEAN,0.241413,0.124605,0.322383,0.008741,0.173930,0.184224
DLT_SUM,0.558978,0.359685,0.511884,0.140932,0.356778,0.489581
IDT_DLT_MAX,0.490350,0.315581,0.430658,0.028920,0.319495,0.402853
IDT_DLT_MEAN,0.531198,0.374326,0.377184,0.001553,0.372398,0.336026
IDT_DLT_SUM,0.621436,0.428857,0.460884,0.147549,0.400523,0.504203
LE_MEAN,0.264326,0.206663,0.179775,-0.022889,0.198940,0.169424
